In [ ]:
%load_ext autoreload
%autoreload 2
%gui qt

In [ ]:
import os
import pya

TEST_FILE = os.path.join(os.path.abspath('..'), "data\\stereoTest.wav")
signal = pya.Asig(TEST_FILE)

In [ ]:
from pyavis.backends.qt_v2.axis import AxisQt
from pyavis.backends.qt_v2.track import TrackQt
from pyavis.backends.qt_v2.multitrack import MultiTrackQt

def on_gfx_track_added(m: MultiTrackQt, t: TrackQt):
    ax_bottom = AxisQt("bottom", lambda value: f'{round(value / t.sampling_rate, 2)}')
    t.set_axis(ax_bottom)
    t.set_view_limits(yMin=-1, yMax=1, xMin=0, minYRange=2, maxYRange=2)

    if len(mt.tracks) > 1:
        t.link_track(mt.tracks[0], 'x')
        t.link_track(mt.tracks[0], 'y')

def on_gfx_track_removed(m: MultiTrackQt, t: TrackQt):
    t.link_track(None, 'x')
    t.link_track(None, 'y')

mt = MultiTrackQt()
mt.trackAdded.connect(lambda args: on_gfx_track_added(args[0], args[1]))
mt.trackRemoved.connect(lambda args: on_gfx_track_removed(args[0], args[1]))

mt.show()

In [ ]:
from pyavis.backends.qt_v2 import SelectionQt
from pyavis.graphics import Selection
from pyavis.shared.util import Subject

class SyncedSelections:
    def __init__(self, start: float, end: float):
        self.selections: list[Selection] = []
        self.region = (start, end)

        self.selectionsUpdated = Subject()
        self.selectionAdded = Subject()
        self.selectionRemoved = Subject()

    def add_selection(self) -> Selection:
        selection = SelectionQt(orientation="vertical", region=self.region)
        selection.regionChanged.connect(self._callback)

        self.selections.append(selection)
        self.selectionAdded.emit(selection)

        return selection

    def update_all_selections(self, region: tuple[int, int]):
        self.region = region
        for selection in self.selections:
            selection.set_region(self.region)
    
    def remove_selection(self, selection: Selection):
        to_remove = self.selections.pop(self.selections.index(selection))
        to_remove.regionChanged.disconnect(self._callback)

        self.selections.remove(to_remove)
        self.selectionRemoved.emit(to_remove)

    def _callback(self, args):
        self.update_all_selections(args[1])

In [ ]:
from pyavis.shared.multitrack import MultiTrack, Track

track2gfx_track = {}
gfx_track2track = {}

sig2gfx_sig = {}
gfx_sig2sig = {}

def on_track_added(track: Track, mt: MultiTrackQt):
    gfx_track = mt.add_track(track.label, track.sampling_rate)
    track2gfx_track[track] = gfx_track
    gfx_track2track[gfx_track] = track

    def on_signal_added(args, gfx_track):
        gfx_sig = gfx_track.add_signal((args[1][0], 0), args[1][1].signal())
        sig2gfx_sig[args[1][1]] = gfx_sig
        gfx_sig2sig[gfx_sig] = args[1][1]

    def on_signal_removed(args, gfx_track):
        sig = args[1][1]
        gfx_sig = sig2gfx_sig[sig]
        gfx_track.remove(gfx_sig)

        sig2gfx_sig.pop(sig)
        gfx_sig2sig.pop(gfx_sig)    

    track.signalAdded.connect(lambda args, gfx_track=gfx_track: on_signal_added(args, gfx_track))
    track.signalRemoved.connect(lambda args, gfx_track=gfx_track: on_signal_removed(args, gfx_track))

def on_track_removed(track: Track, mt: MultiTrackQt):
    gfx_track = track2gfx_track[track]
    track2gfx_track.pop(track)
    gfx_track2track.pop(gfx_track)

    mt.remove_track(gfx_track)


multi = MultiTrack()
multi.onTrackAdded.connect(lambda args: on_track_added(args[1], mt))
multi.onTrackRemoved.connect(lambda args: on_track_removed(args[1], mt))

In [ ]:
from pyavis.shared.signal import AudioSignal

t_1 = Track("test_1", 44100)
t_2 = Track("test_2", 44100)
t_3 = Track("test_3", 44100)

multi.add_track(t_1)
multi.add_track(t_2)
multi.add_track(t_3)

t_1.try_add(0, AudioSignal(signal[:,0]))
t_2.try_add(0, AudioSignal(signal[:,1]))

In [ ]:
selector = SyncedSelections(0,44000)

sel_1 = selector.add_selection()
sel_2 = selector.add_selection()

mt.tracks[0].set_selection(sel_1)
mt.tracks[1].set_selection(sel_2)

selected = []
for t in mt.tracks:
    selected.append(False if t.get_selection() is None else True)

In [ ]:
import pya
pya.startup(device=index)

In [ ]:
from itertools import compress
import numpy as np

selected_tracks = list(compress(multi.tracks, selected))
sigs = np.array([t[int(selector.region[0]):int(selector.region[1])] for t in selected_tracks]).T

asig = pya.Asig(sigs, channels=sigs.shape[1])
asig.play()